# Laboratorio 6

#### 20880 Sebastian Aristondo
#### 20293 Daniel Gonzalez

## 2. Carga de datos

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.corpus import opinion_lexicon
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
import datetime
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
from langdetect import detect

In [ ]:
data = pd.read_csv('traficogt.csv', sep=',')
data.head()

## 3. Limpieza y preprocesamiento de datos

El dataset tiene los datos crudos, por lo que debemos de limpiarlos y preprocesarlos para poder trabajar con ellos. Para esto, se utilizo la libreria pandas para poder leer el archivo csv y poder trabajar con el. Luego utilizaremos varias funciones de nltk para poder limpiar los datos y dejarlos listos para poder trabajar con ellos.

In [ ]:
def remove_urls(rawContent):
    url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    return re.sub(url_pattern, '', rawContent)

Se realizaron tres acciones iniciales para limpiar y preprocesar datos. Primero se eliminó cualquier URL de los datos usando una expresión regular. También se mantuvieron solamente los caracteres que estuvieran de la a a la z, mayúsculas o minúsculas. Esto quiere decir que se quitó cualquier caracter como "#" o "@" y signos de puntuación. Por otra parte, se pasaron todas las palabras a minúsculas.

In [ ]:
def remove_tildes(rawContent):
    rawContent = rawContent.replace('á', 'a')
    rawContent = rawContent.replace('é', 'e')
    rawContent = rawContent.replace('í', 'i')
    rawContent = rawContent.replace('ó', 'o')
    rawContent = rawContent.replace('ú', 'u')
    return rawContent

In [ ]:
data['rawContent'] = data['rawContent'].apply(remove_urls)
data['rawContent'] = data['rawContent'].apply(lambda x: x.lower())
data['rawContent'] = data['rawContent'].apply(remove_tildes)

In [ ]:
data['rawContent'] = data['rawContent'].str.replace('[^a-zA-Z0-9]', ' ', regex=True)

Se removieron URLS y se quitaron caracteres especiales. También se pasaron todos los tweets a minúsculas y se removieron las tildes, para poder tener una forma estandarizada de los símbolos de los tweets en el dataset.

In [ ]:
def remove_stop_words(sentence):
    stop_words = set(stopwords.words('spanish'))  
    words = sentence.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    new_sentence = ' '.join(filtered_words)
    return new_sentence

data['rawContent_clean'] = data['rawContent'].apply(remove_stop_words)

Se eliminaron las stop words en español que pudiera tener el dataset.

In [ ]:
data['rawContent_clean'].head()

In [ ]:
def lemmatize_words(words):
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return lemmatized_words

data['rawContent_lemmatized'] = data['rawContent_clean'].apply(lambda x: lemmatize_words(x.split()))
data['rawContent_lemmatized_text'] = data['rawContent_lemmatized'].apply(lambda x: ' '.join(x))

Con el objetivo de analizar de una manera más sencilla los tweets los lematizaremos para poder obtener palabras clave como zona y lluvia.


In [ ]:
data['rawContent_lemmatized'].head()

In [ ]:
data['rawContent_lemmatized_text'].head()

In [ ]:
data = data.drop("number", axis=1)   

In [ ]:
def delete_non_spanish_tweets(texto):
    try:
        return detect(texto) == 'es'
    except:
        # Si no se puede detectar el idioma, se asume que no es español
        return False

In [ ]:
data = data[data['rawContent'].apply(delete_non_spanish_tweets)]

## 4. Análisis exploratorio

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
nas_coordenadas = data['coordinates'].isna().sum()
print('Porcentaje de tweets sin coordenadas: ', nas_coordenadas/data.shape[0]*100)

print("Cantidad de tweets con coordenadas: ", data.shape[0] - nas_coordenadas)


Como se puede observar, casi todos los tweets no tienen coordenadas, por lo tanto, no es posible realizar un análisis de tweets por ubicación para determinar que áreas tienen más tráfico.

In [ ]:
coincidencias = []

# Utilizar expresiones regulares para encontrar coincidencias de "zona" seguida de un número
pattern = r'\bzona\s+(\d+)\b'  # \b asegura que "zona" sea una palabra completa, \s+ coincide con uno o más espacios, \d+ coincide con uno o más dígitos

# Buscar coincidencias en la columna 'texto' y almacenarlas en la lista
for texto in data['rawContent_clean']:
    matches = re.findall(pattern, texto)
    coincidencias.extend(matches)

In [ ]:
# Contar la frecuencia de cada número
conteo_coincidencias = Counter(coincidencias)

# Obtener las etiquetas (números) y sus frecuencias
etiquetas = list(conteo_coincidencias.keys())
frecuencias = list(conteo_coincidencias.values())

# Crear el gráfico de barras
plt.bar(etiquetas, frecuencias)

# Agregar etiquetas y título
plt.xlabel('Número')
plt.ylabel('Frecuencia')
plt.title('Frecuencia de coincidencias por número')

# Mostrar el gráfico
plt.show()

Se puede observar que de los tweets relacionados al tráfico, la mayor cantidad vienen de zona 1, zona 10, zona 11 y zona 18. 

In [ ]:
usuarios = data['user'].unique()
print('Cantidad de usuarios unicos en el dataset: ', len(usuarios))

In [ ]:
def extraer_valor(diccionario):
    diccionario = eval(diccionario)
    return diccionario["username"]

data["username"]=data["user"].apply(extraer_valor)


In [ ]:
tweets_mas_rt = data.groupby('username')['retweetCount'].sum()
tr_count_df = tweets_mas_rt.reset_index()
tr_count_df = tr_count_df.rename(columns={'retweetCount': 'count'})
tr_count_df = tr_count_df.sort_values(by='count', ascending=False)
tr_count_df = tr_count_df.head(10)

plt.figure(figsize=(10, 6))  # Tamaño del gráfico
plt.bar(tr_count_df['username'], tr_count_df['count'])
plt.xlabel('Usuario')
plt.ylabel('Conteo de Retweets')
plt.title('Conteo de Retweets por Usuario')
plt.xticks(rotation=90)  # Rotar las etiquetas del eje x para una mejor visualización
plt.show()


In [ ]:
print(f"Como podemos ver el usuario {tr_count_df.iloc[0]['username']} pareciera ser un usuario muy activo en Twitter en cuanto a tweets \nrelacionados al tráfico. Mucha gente comparte la información que el público. Este hallazgo es interesante ya que asumimos \nque el mayor referente sobre este rubro sería prensa libre, Amílcar Montejo o algún otro medio de comunicación.")


In [ ]:
tweets_mas_likes = data.groupby('username')['likeCount'].sum()
tr_count_df_likes = tweets_mas_likes.reset_index()
tr_count_df_likes = tr_count_df_likes.rename(columns={'likeCount': 'count'})
tr_count_df_likes = tr_count_df_likes.sort_values(by='count', ascending=False)
tr_count_df_likes = tr_count_df_likes.head(10)

plt.figure(figsize=(10, 6))  # Tamaño del gráfico
plt.bar(tr_count_df['username'], tr_count_df_likes['count'])
plt.xlabel('Usuario')
plt.ylabel('Conteo de Retweets')
plt.title('Conteo de Retweets por Usuario')
plt.xticks(rotation=90)  # Rotar las etiquetas del eje x para una mejor visualización
plt.show()

In [ ]:
frecuencia_usuarios = data['username'].value_counts()

usuarios_ordenados = frecuencia_usuarios.sort_values(ascending=False)

# Tomar los 10 usuarios más frecuentes
top_10_usuarios = usuarios_ordenados.head(10)

# Crear un gráfico de barras
plt.figure(figsize=(15, 6))  # Tamaño del gráfico
top_10_usuarios.plot(kind='bar')
plt.xlabel('Usuario')
plt.ylabel('Frecuencia')
plt.title('Frecuencia de Usuarios')
plt.xticks(rotation=0)  # Rotar las etiquetas del eje x para una mejor visualización
plt.show()

In [ ]:
# Cargar el modelo pre-entrenado de BERT para análisis de sentimiento
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Crear una función para realizar análisis de sentimiento
def analyze_sentiment(text):
    # Tokenizar el texto y obtener la salida del modelo
    inputs = tokenizer(text, return_tensors='pt')
    outputs = model(**inputs)

    # Obtener la predicción de sentimiento
    prediction = torch.argmax(outputs.logits, dim=1).item()
    
    # Definir la escala de sentimiento
    sentiment_scale = {
        0: 'Muy negativo',
        1: 'Negativo',
        2: 'Neutral',
        3: 'Positivo',
        4: 'Muy positivo'
    }
    
    # Obtener la etiqueta de sentimiento
    sentiment_label = sentiment_scale[prediction]
    
    return sentiment_label


In [ ]:
def verificar_sentimiento(dataframe, columna):

    for valor in dataframe[columna]:
        filas = data[data['username'] == valor]
        result = filas['rawContent_clean'].apply(analyze_sentiment)
        frecuencia_valores = result.value_counts()
        valor_mas_comun = frecuencia_valores.idxmax()
        print("Los tweets del usuario ", valor, " son mayormente ", valor_mas_comun)
    

In [ ]:
print("Los sentimientos de los tweets de los usuarios que tienen más retweets son:")
verificar_sentimiento(tr_count_df, "username")

In [ ]:
print("Los sentimientos de los tweets de los usuarios que tienen más likes son:")
verificar_sentimiento(tr_count_df_likes, "username")

*Pone comentario sobre la negatividad xd
ARREGLAR QUE NO SE VAYA A QUEDAR ASI

In [ ]:
palabras = ['calzada la paz', 'zona 5', 'hundimiento']
patron = '|'.join(palabras)
resultados = data[data['rawContent_lemmatized_text'].str.contains(patron, case=False, na=False)]['rawContent_lemmatized_text']


In [ ]:
df_lluvia = data[data['rawContent_lemmatized_text'].str.contains('lluvia', case=False)]
df_lluvia.shape